In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import plotly.express as px

In [2]:
from scripts.preproc import load_data, split_df, get_xxyy
from scripts.train import train_rfr, predict_val
from scripts.submit import get_submitable
train_cols = ['precipitation_amt_mm', 'weekofyear_sin', 'weekofyear_cos']
target_col = 'total_cases'
test_size = 0

# 1.Preprocessing

## 1.1 Train features and labes are merged, both train and test DFs are devided by cities

In [3]:
train_sj, train_iq, test_sj, test_iq = load_data()

In [ ]:
# train_sj.head().T

## 1.2. Visualisation

In [ ]:
# fig = px.line(train_sj, x='date', y=["precipitation_amt_mm", 'station_avg_temp_c', 'total_cases', 'weekofyear_sin', 'weekofyear_cos'], title='San Juan')
# fig.show()
# fig = px.line(train_iq, x='date', y=["precipitation_amt_mm", 'station_avg_temp_c', 'total_cases', 'weekofyear_sin', 'weekofyear_cos'], title='Iquitos')
# fig.show()

## 1.3. Spliting into training and validation sets

I suggest that at the end of preprocessing for either city we get **one** dictionary with **four** dataframes: X_train, X_val, y_train, y_val

In [5]:
xxyy_sj = get_xxyy(train_sj, test_size, train_cols, target_col)
xxyy_iq = get_xxyy(train_iq, test_size, train_cols, target_col)

Train size: 936, validation size: 0
Train size: 520, validation size: 0


# 2. Training

At the end of training we need:
1. a model compatible with scikit-learn api
2. (maybe???) an encoder

In [6]:
from sklearn.model_selection import TimeSeriesSplit

cv = TimeSeriesSplit(5)
scoring = 'neg_mean_absolute_error'

In [7]:
from sklearn.ensemble import RandomForestRegressor

estimator = RandomForestRegressor()
params = dict(
	n_estimators=[700, 1000, 1500, 2000],
	max_depth=[2, 3, 4, 5, 10]
)

In [8]:
from xgboost import XGBRegressor
# https://machinelearningmastery.com/xgboost-for-regression/

In [16]:
estimator = RandomForestRegressor()
params = dict(
	n_estimators = [2500, 2000, 1500],
	max_depth = [3, 4, 5],
	# eta = [0.3, 0.1, 0.01],
	# subsample = [0.3, 0.5, 0.8, 1],
	# colsample_bytree = 1,
)

In [18]:
model_sj = train_grid(xxyy_sj, estimator, params, scoring)
model_iq = train_grid(xxyy_iq, estimator, params, scoring)

In [19]:
model_sj.best_params_, model_iq.best_params_

({'max_depth': 3, 'n_estimators': 2000},
 {'max_depth': 3, 'n_estimators': 2000})

In [20]:
# y_pred_sj = predict_val(model_sj, xxyy_sj['X_val'], xxyy_sj['y_val'])
# y_pred_iq = predict_val(model_iq, xxyy_iq['X_val'], xxyy_iq['y_val'])

# 3. Predict for test features and format for submition

For that we need:
1. [from precrocessing] - (list of) test DFs
2. [from training]      - (list of) models

In [21]:
get_submitable([test_sj, test_iq], [model_sj, model_iq], train_cols)

In [ ]:
# target['date'] = pd.to_datetime(target['year'].astype('str') + '-' + target['weekofyear'].astype('str') + '-0', format='%Y-%W-%w')